# 動画から人物のみを検出して画像として保存する

作業フォルダに移動する

In [2]:
cd '/content/drive/My Drive/AI/security_detect'

/content/drive/My Drive/AI/security_detect


フォルダ構成を確認。「security_train.ipynb」「face」「noface」があればOK

In [28]:
ls

face/  noface/  security_image.ipynb  security_train.ipynb
img/   result/  security.pkl          thief.mp4


人物画像と、それ以外を判別させる。

アルゴリズムはランダムフォレストを使用。

In [13]:
import cv2
import os, glob
import joblib
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


# 画像の学習サイズやパスを指定
image_size = (64, 32)

#作業フォルダのパスを指定
path = '/content/drive/My Drive/AI/security_detect'
path_fish = path + '/face'
path_nofish = path + '/noface'
x = [] # 画像データ
y = [] # ラベルデータ

# 画像データを読み込んで配列に追加 
def read_dir(path, label):
    files = glob.glob(path + "/*.jpg")
    for f in files:
        img = cv2.imread(f)
        img = cv2.resize(img, image_size)
        img_data = img.reshape(-1, ) # 一次元に展開
        x.append(img_data)
        y.append(label)

# 画像データを読み込む
read_dir(path_nofish, 0)
read_dir(path_fish, 1)

# データを学習用とテスト用に分割する
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# データを学習 
clf = RandomForestClassifier()
clf.fit(x_train, y_train)

# 精度の確認 
y_pred = clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

# データを保存 
joblib.dump(clf, 'security.pkl')


1.0


['security.pkl']

学習結果が「security.pkl」として保存される。

この学習結果を元に、動画から人物判定を実施する。

In [23]:
import cv2, os, copy

import joblib

# 学習済みデータを取り出す
clf = joblib.load("security.pkl")
output_dir = "./result"
img_last = None # 前回の画像
secure_th = 0.5 # 画像を出力するかどうかのしきい値
count = 0
frame_count = 0
if not os.path.isdir(output_dir): os.mkdir(output_dir)

# 動画ファイルから入力を開始 
cap = cv2.VideoCapture("thief.mp4")
while True:
    # 画像を取得
    is_ok, frame = cap.read()
    if not is_ok: break
    # frame = cv2.resize(frame, (640, 360))
    frame2 = copy.copy(frame)
    frame_count += 1
    # 前フレームと比較するために白黒に変換
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (15, 15), 0)
    img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]
    if not img_last is None:
        # 差分を得る
        frame_diff = cv2.absdiff(img_last, img_b)
        cnts = cv2.findContours(frame_diff, 
            cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)[0]
        # 差分領域に人物が映っているか調べる
        person_count = 0
        for pt in cnts:
            x, y, w, h = cv2.boundingRect(pt)
            if w < 29 or w > 500: continue # ノイズを除去
            # 抽出した領域に人物が映っているか確認 
            imgex = frame[y:y+h, x:x+w]
            imagex = cv2.resize(imgex, (64, 32))
            image_data = imagex.reshape(-1, )
            pred_y = clf.predict([image_data]) 
            if pred_y[0] == 1:
                person_count += 1
                cv2.rectangle(frame2, (x, y), (x+w, y+h), (0,255,0), 2)
        # 人物が映っているか？
        if person_count > secure_th:
            fname = output_dir + "/person" + str(count) + ".jpg"
            cv2.imwrite(fname, frame)
            count += 1
        img_last = img_b
cap.release()
cv2.destroyAllWindows()
print("ok", count, "/", frame_count)



ok 43 / 211


成功すると、「result」というフォルダが生成され、中には人物が映った部分のみが保存されている。